In [56]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Plotting
import matplotlib.pyplot as plt

# Data science
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold

# Geospatial
import contextily as cx
from shapely.geometry import Point, Polygon
import xarray as xr
import rasterio.features
import rasterio as rio

# API
import requests
import json

# Import Planetary Computer
import fsspec
import stackstac
import pystac
import pystac_client
import planetary_computer
import pickle

In [58]:
def get_Dataset():
    weather = xr.load_dataarray("assets/weatherData_Aus.nc")
    species = pd.read_csv("assets/finalDataset_AllFrogs.csv").iloc[:,1:]
    species = species.dropna()

    print("Australia weather data loaded...\nWeather data shape: ",weather.shape)
    print("Species data loaded...\nFrog data shape: ",species.shape)
    cols = ['decimalLatitude', 'decimalLongitude', 'ppt_mean', 'soil_mean','tmax_mean','tmin_mean']
    temp = species[cols]
    print(temp.columns)

    col1 = ['decimalLatitude', 'decimalLongitude']
    temp1 = species[col1]
    test = temp.iloc[:]
    test1 = temp1.iloc[:]   

    return weather, test, test1, species

def get_model():
    print("Model loaded ...")
    full_model = pickle.load(open("assets/randomforest_allfrog.pkl", 'rb'))
    print(full_model)
    return full_model

def plot_graphs(weather, test1, target_species="Crinia signifera"):
    nrow = 2
    ncol = 2
    fig, ax = plt.subplots(nrow, ncol, figsize=(13, 7), sharex=True, sharey=True)

    bands = weather.band.values
    filt = test1.prediction == 1
    cmaps = ["cool", "cool", "Blues", "BrBG"]

    for i in range(len(bands)):
        xr.plot.imshow(weather[i], 'x', 'y', cmap=cmaps[i], ax=ax[i//ncol, i%ncol]) 
        ax[i//ncol, i%ncol].set_title(bands[i])
        ax[i//ncol, i%ncol].scatter(test1[filt].decimalLongitude, test1[filt].decimalLatitude,
                                    color = 'green', marker='.', alpha=0.5,
                                    label=target_species if i==0 else ''
                                    )

    fig.suptitle("Spatial distribution of Terraclimate variables", fontsize=20)
    fig.legend(loc=(0.85, 0.933))

# def predict_frogs(predictor_image, model):
#     """Returns a (1, n, m) xarray where each pixel value corresponds to the probability of a frog occurrence.
    
#     Takes in the multi-band image outputted by the `create_predictor_image` function as well as the
#     trained model and returns the predictions for each pixel value. Firstly, the $x$ and $y$ indexes
#     in the predictor image are stacked into one multi-index $z=(x, y)$ to produce an $k\times n$
#     array, which is the format required to feed into our logistic regression model. Then, the array
#     is fed into the model, returning the model's predictions for the frog likelihood at each pixel. 
#     The predicted probabilities are then indexed by the same multi-index $z$ as before, which allows 
#     the array to be unstacked and returned as a one-band image, ready for plotting.

#     Arguments:
#     predictor_image -- (K, n, m) xarray, where K is the number of predictor variables.
#     model -- sklearn model with K predictor variables.
#     """
#     # Stack up pixels so they are in the appropriate format for the model
#     predictor_image = predictor_image.stack(z=("y", "x")).transpose()
#     # Reorder variables to be in same order as model
#     predictor_image = predictor_image.sel(band=model.feature_names_in_)
#     # Location of null values so that we can skip them (prediction model will break if nulls are present)
#     null_pixels = (np.sum(predictor_image.isnull(), axis=-1) > 0)
#     # Empty probabilities array
#     probabilities = np.zeros((len(null_pixels), 2))
#     # Calculate probability for each non-null pixel point
#     probabilities[~null_pixels] = model.predict_proba(
#         predictor_image[~null_pixels]
#     )
#     probs = probabilities[~null_pixels][:,1]
#     Good = 0
#     Normal = 0
#     Total = 0
#     for value in probs: 
#         if value >= 0.6:
#             Good+=1
#         elif value>=0.4 and value<0.6:
#             Normal+=1
#         Total+=1
#     GoodScore = Good/Total
#     NormalScore = Normal/Total
#     print("Random Score: ", GoodScore/NormalScore)
#     print("Goodness score: ",GoodScore)
#     print("Normalness score: ",NormalScore)


#     # Set null pixels to a probability of null
#     probabilities[null_pixels] = np.array([np.nan, np.nan])
#     # Just take probability of frog (class=1)
#     probabilities = probabilities[:,1]


#     # Add the coordinates to the probabilities, saving them in an xarray
#     resultant_image = xr.DataArray(
#         data=probabilities,
#         dims=['z'],
#         coords=dict(
#             z=predictor_image.z
#         )
#     )
#     # Unstack the image
#     resultant_image = resultant_image.unstack()
#     return resultant_image

def save_to_geotiff(arr, filename, bbox, dtype='uint8', verbose=False):
    # If filename is specified, save mosaic to geotiff  
    bands = arr.shape[0]
    height = arr.shape[1]
    width = arr.shape[2]
        
    # write bandnames to separate file
    with open(filename+'.bands', 'w') as file:
        file.write(','.join(arr.band.values))
        
    min_lon, min_lat, max_lon, max_lat = bbox
    
    # Define the Coordinate Reference System (CRS) to be common Lat-Lon coordinates
    # Define the tranformation using our bounding box so the Lat-Lon information is written to the GeoTIFF
    gt = rasterio.transform.from_bounds(min_lon, min_lat, max_lon, max_lat, width, height)
    arr.rio.write_crs("epsg:4326", inplace=True)
    arr.rio.write_transform(transform=gt, inplace=True);
    
    # Create the GeoTIFF output file using the defined parameters
    with rasterio.open(filename+'.tiff', 'w', driver='GTiff', width=width, height=height, crs='epsg:4326',
                       transform=gt, count=bands, compress='lzw', dtype=dtype) as dst:
        for band in range(bands):
            print(f'writing {band+1} of {bands}') if verbose else None
            dst.write(arr[band],band+1)
        dst.close()

# Helper function to assist with memory - sometimes predicting the entire scene at once crashes the kernel
def predict_in_batches(weather, model, batches=10):
    probabilities = np.zeros((weather.shape[0], len(model.classes_)))
    offset=0
    for batch in np.array_split(weather,batches):
        probabilities[offset:(offset+batch.shape[0])] = model.predict_proba(
            batch
        )
        offset += batch.shape[0]
    return probabilities

def predict_frogs(predictor_image, model):
    """Returns a (k, n, m) xarray where each pixel value corresponds to the probability of a frog occurrence.
    
    Takes in the multi-band image outputted by the `create_predictor_image` function as well as the
    trained model and returns the predictions for each pixel value. Firstly, the $x$ and $y$ indexes
    in the predictor image are stacked into one multi-index $z=(x, y)$ to produce an $k\times n$
    array, which is the format required to feed into our logistic regression model. Then, the array
    is fed into the model, returning the model's predictions for the frog likelihood at each pixel. 
    The predicted probabilities are then indexed by the same multi-index $z$ as before, which allows 
    the array to be unstacked and returned as a one-band image, ready for plotting.

    Arguments:
    predictor_image -- (K, n, m) xarray, where K is the number of predictor variables.
    model -- sklearn model with K predictor variables.
    """
    # Take only bands in model
    predictor_image = predictor_image.sel(band=model.feature_names_in_)
    # Stack up pixels so they are in the appropriate format for the model
    predictor_image = predictor_image.stack(z=("y", "x")).transpose()
    # Location of null values so that we can skip them (prediction model will break if nulls are present)
    null_pixels = (np.sum(predictor_image.isnull(), axis=-1) > 0)
    # Empty probabilities array
    probabilities = np.zeros((len(null_pixels), len(model.classes_)))
    # Calculate probability for each non-null pixel point (in batches to preserve memory)
    probabilities[~null_pixels] = predict_in_batches(predictor_image[~null_pixels], model, batches=10)
    probabilities[null_pixels] = np.array([np.nan]*len(model.classes_))
    # Add the coordinates to the probabilities, saving them in an xarray
    resultant_image = xr.DataArray(
        data=probabilities,
        dims=['z', 'band'],
        coords=dict(
            z=predictor_image.z,
            band=model.classes_
        )
    )
    # Unstack the image
    resultant_image = resultant_image.unstack()
    return resultant_image
# s = ['Austrochaperina pluvialis','Crinia glauerti','Crinia signifera','Litoria fallax','Ranoidea australis']


def plot_heatmap(weather, model, frog_data, species_names = ['Austrochaperina pluvialis','Crinia glauerti','Crinia signifera','Litoria fallax','Ranoidea australis'], 
                 crs = {'init':'epsg:4326'}):
    """Plots a real map, probability heatmap, and model classification regions for the probability image from our model.

    Arguments:
    resultant_image -- (1, n, m) xarray of probabilities output from the model
    frog_data -- Dataframe of frog occurrences, indicated with a 1 in the occurrenceStatus column. 
                 Must contain ["occurrenceStatus", "decimalLongitude", "decimalLatitude"]
    title -- string that will be displayed as the figure title
    crs -- coordinate reference system for plotting the real map. Defaults to EPSG:4326.
    """
    resultant_image = predict_frogs(weather, model)
    storage_path = 'assets/'
    bbox = (152.4, -31.9, 152.9, -31.4)
    save_to_geotiff(resultant_image, f'{storage_path}_result', bbox , dtype='float32', verbose=False)
    fig, ax = plt.subplots(len(species_names), 3, figsize=(20, 7*len(species_names)), sharey=True, sharex=True)
    extent = [resultant_image.x.min(),resultant_image.x.max(),resultant_image.y.min(),resultant_image.y.max()]
    max_val = resultant_image.values.max()
    min_val = resultant_image.values.min()
    cmap = 'Greens'
    regions_image = resultant_image.argmax(dim='band')
    for i, species in enumerate(species_names):
        species_plot = resultant_image.sel(band=species)
        # Plot real map
        ax[i, 0].scatter(
            x=[extent[0], extent[1]], y=[extent[2], extent[3]], alpha=0
        )
        cx.add_basemap(ax[i, 0], crs=crs)
        ax[i, 0].set_title(f'Real map\n{title}')        
        # Plot heatmap from model
        heatmap = species_plot.plot.imshow(
            x='x', y='y', ax=ax[i, 1], cmap=cmap, vmin=min_val, vmax=max_val, interpolation='none', add_colorbar=False
        )
        ax[i, 1].set_aspect('equal')
        ax[i, 1].set_title(f'{species}\nModel Probability Heatmap')
        # Plot binary classification from model
        regions = xr.where(regions_image.isnull(),np.nan, regions_image==i).plot.imshow(
            x='x', y='y', ax=ax[i, 2], cmap=cmap, vmin=min_val, vmax=max_val, interpolation='none', add_colorbar=False
        )
        ax[i, 2].set_aspect('equal')
        ax[i, 2].set_title(f'{species}\nModel Classification Regions')
        # Plot real frogs
        for j in range(3):
            filt = frog_data.species == species
            ax[i, j].scatter(
                frog_data[filt].decimalLongitude, frog_data[filt].decimalLatitude, 
                color = 'dodgerblue', marker='.', alpha=0.5, label=species if j==2 else ''
            )
        ax[i, 2].legend()
    # fig.colorbar(heatmap, ax=ax, location = 'bottom', aspect=40)
    # fig.suptitle(title, x=0.5, y=0.9, fontsize=20)
    fig.tight_layout()
    plt.show()
    
# def plot_heatmap(weather, test1, model, crs = {'init':'epsg:4326'}):
#     resultant_image = predict_frogs(weather, model)
#     # resultant_image.to_netcdf("resultant.nc")

#     fig, ax = plt.subplots(1, 3, figsize = (20, 10), sharex=True, sharey=True)
#     extent = [resultant_image.x.min(),resultant_image.x.max(),resultant_image.y.min(),resultant_image.y.max()]
#     cmap = 'PiYG'

#     # Plot real map
#     ax[0].scatter(x=[extent[0], extent[1]], y=[extent[2], extent[3]], alpha=0)
#     cx.add_basemap(ax[0], crs=crs)
#     ax[0].set_title('Real map')
    
#     # Plot heatmap from model
#     heatmap = resultant_image.plot.imshow(
#         x='x', y='y', ax=ax[1], cmap=cmap, vmin=0, vmax=1, interpolation='none', add_colorbar=False
#     )
#     ax[1].set_aspect('equal')
#     ax[1].set_title('Model Probability Heatmap')

#     # Plot binary classification from model
#     regions = xr.where(resultant_image.isnull(), np.nan, resultant_image>0.5).plot.imshow(
#             x='x', y='y', ax=ax[2], cmap=cmap, vmin=0, vmax=1, interpolation='none', add_colorbar=False
#     )
#     ax[2].set_aspect('equal')
#     ax[2].set_title('Model Classification Regions')

#     # Plot real frogs
#     for i, axis in enumerate(ax):
#         filt = test1.prediction == 1
#         axis.scatter(
#             test1[filt].decimalLongitude, test1[filt].decimalLatitude, 
#             color = 'dodgerblue', marker='.', alpha=0.5, label='Target Species' if i==0 else ''
#         )

#     fig.colorbar(heatmap, ax=ax, location = 'bottom', aspect=40)
#     fig.legend(loc = (0.9, 0.9))
#     # fig.suptitle(title, x=0.5, y=0.9, fontsize=20)


def main():
    # Sort acc to country -> Australia
    weather, test, test1, species = get_Dataset()
    model = get_model()

    # predictions
    testData = test.drop(['decimalLongitude', 'decimalLatitude'], 1)
    print(testData.columns)

    OccurPredictions = model.predict(testData)
    print(OccurPredictions)
    
    # test1['prediction'] = OccurPredictions
    print("Prediction done...")
    
    # Plot results
    plot_heatmap(weather, model, species)
    plot_graphs(weather, OccurPredictions)


main()


Australia weather data loaded...
Weather data shape:  (4, 1024, 1024)
Species data loaded...
Frog data shape:  (148136, 15)
Index(['decimalLatitude', 'decimalLongitude', 'ppt_mean', 'soil_mean',
       'tmax_mean', 'tmin_mean'],
      dtype='object')
Model loaded ...
RandomForestClassifier()
Index(['ppt_mean', 'soil_mean', 'tmax_mean', 'tmin_mean'], dtype='object')
[[0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 ...
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]]
Prediction done...


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 2.

In [37]:
a = xr.load_dataarray("resultant.nc")



<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.0
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.0
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.1
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.1
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.2
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.2
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.2
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.3
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        float64 115.3
<xarray.DataArray ()>
array(nan)
Coordinates:
    y        float64 -43.61
    x        floa

KeyboardInterrupt: 